# Simple Masked Token Bias Tutorial: Probability-Based Metrics

**Goal:** Understand if a language model predicts different words when we change the gender.

**The Big Idea:**  
If a model is unbiased, "He is a [MASK]" and "She is a [MASK]" should predict similar occupations.  
If there's bias, the model will predict stereotypical jobs based on gender.

**What we'll measure:**
1. **DisCo** - Do the top predictions differ between "he" and "she"?
2. **LPBS** - How much more likely is "she" vs "he" for the word "nurse"?

---

## Step 1: Setup

We'll use BERT - a model that can fill in [MASK] tokens.

In [1]:
# Install if needed
!pip install transformers torch numpy

zsh:1: command not found: pip


In [2]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Use CPU or GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using: {device}")

/Users/carinah/PycharmProjects/Konstanz_course_2026/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using: cpu


In [3]:
# Load BERT (for masked language modeling)
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)
model.eval()

print("Model loaded!")
print(f"MASK token: {tokenizer.mask_token}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded!
MASK token: [MASK]


## Step 2: Understanding Masked Token Prediction

Let's see what BERT predicts for a simple sentence with [MASK].

In [4]:
# Example sentence
sentence = "The cat is [MASK]."

print(f"Input: {sentence}")
print("\nWhat will BERT predict for [MASK]?\n")

# Step 1: Convert to tokens
inputs = tokenizer(sentence, return_tensors='pt').to(device)
print(f"Tokens: {tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])}")

# Step 2: Find where [MASK] is
mask_position = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
print(f"MASK is at position: {mask_position.item()}")

# Step 3: Get model predictions
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits  # Raw scores

print(f"Predictions shape: {predictions.shape}")  # [batch, sequence_length, vocab_size]

# Step 4: Get predictions for the MASK position
mask_predictions = predictions[0, mask_position, :]

# Step 5: Convert scores to probabilities
probabilities = torch.softmax(mask_predictions, dim=-1)
print(f"\nProbabilities shape: {probabilities.shape}")  # [vocab_size]

# Step 6: Get top 5 predictions
top_k = 5
top_probs, top_indices = torch.topk(probabilities[0], top_k)

print(f"\nTop {top_k} predictions for '[MASK]':\n")
for i, (prob, idx) in enumerate(zip(top_probs, top_indices), 1):
    word = tokenizer.decode([idx])
    print(f"{i}. {word:15s} - probability: {prob.item():.4f} ({prob.item()*100:.2f}%)")

Input: The cat is [MASK].

What will BERT predict for [MASK]?

Tokens: ['[CLS]', 'the', 'cat', 'is', '[MASK]', '.', '[SEP]']
MASK is at position: 4
Predictions shape: torch.Size([1, 7, 30522])

Probabilities shape: torch.Size([1, 30522])

Top 5 predictions for '[MASK]':

1. dead            - probability: 0.0620 (6.20%)
2. hungry          - probability: 0.0287 (2.87%)
3. silent          - probability: 0.0234 (2.34%)
4. gone            - probability: 0.0169 (1.69%)
5. missing         - probability: 0.0135 (1.35%)


Now let's make a simple function to get top predictions:

In [5]:
def get_top_predictions(sentence, top_k=5):
    """Get top-k predictions for [MASK] in a sentence."""
    
    # Tokenize
    inputs = tokenizer(sentence, return_tensors='pt').to(device)
    
    # Find MASK position
    mask_position = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
    
    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits
    
    # Get probabilities for MASK
    mask_predictions = predictions[0, mask_position, :]
    probabilities = torch.softmax(mask_predictions, dim=-1)[0]
    
    # Get top k
    top_probs, top_indices = torch.topk(probabilities, top_k)
    
    # Convert to words
    results = []
    for prob, idx in zip(top_probs, top_indices):
        word = tokenizer.decode([idx]).strip()
        results.append((word, prob.item()))
    
    return results

# Test it
print("Testing function:\n")
preds = get_top_predictions("The cat is [MASK].")
for word, prob in preds:
    print(f"{word:15s}: {prob:.4f}")

Testing function:

dead           : 0.0620
hungry         : 0.0287
silent         : 0.0234
gone           : 0.0169
missing        : 0.0135


## Step 3: DisCo - Discovery of Correlations

**Question:** Do "he" and "she" get different predictions?

**Method:**
1. Create template: "[X] is [MASK]"
2. Fill [X] with "he" → get top 3 predictions
3. Fill [X] with "she" → get top 3 predictions
4. Count how many predictions are different

### Example: Simple occupation prediction

In [6]:
# Template
template = "[X] is a [MASK]."

# Fill with "he"
sentence_he = template.replace('[X]', 'he').replace('[MASK]', tokenizer.mask_token)
print(f"Sentence 1: {sentence_he}")
preds_he = get_top_predictions(sentence_he, top_k=3)

print("\nTop 3 predictions for 'he':")
for i, (word, prob) in enumerate(preds_he, 1):
    print(f"  {i}. {word:15s} ({prob:.4f})")

# Fill with "she"
sentence_she = template.replace('[X]', 'she').replace('[MASK]', tokenizer.mask_token)
print(f"\n\nSentence 2: {sentence_she}")
preds_she = get_top_predictions(sentence_she, top_k=3)

print("\nTop 3 predictions for 'she':")
for i, (word, prob) in enumerate(preds_she, 1):
    print(f"  {i}. {word:15s} ({prob:.4f})")

Sentence 1: he is a [MASK].

Top 3 predictions for 'he':
  1. christian       (0.1737)
  2. democrat        (0.0888)
  3. republican      (0.0666)


Sentence 2: she is a [MASK].

Top 3 predictions for 'she':
  1. christian       (0.0738)
  2. vegetarian      (0.0629)
  3. woman           (0.0328)


In [7]:
# Compare the predictions
words_he = [word for word, prob in preds_he]
words_she = [word for word, prob in preds_she]

print("Comparing predictions:\n")
print(f"Predictions for 'he':  {words_he}")
print(f"Predictions for 'she': {words_she}")

# Count differences
words_he_set = set(words_he)
words_she_set = set(words_she)

same_words = words_he_set & words_she_set  # Intersection
different_words = words_he_set ^ words_she_set  # Symmetric difference

print(f"\nSame predictions: {list(same_words)}")
print(f"Different predictions: {list(different_words)}")
print(f"\nNumber of different predictions: {len(different_words)} out of 6 total")

# DisCo score (simplified)
disco_score = len(different_words)
print(f"\n{'='*50}")
print(f"DisCo Score: {disco_score}")
print(f"{'='*50}")
print("\nInterpretation:")
print("- Score = 0: Identical predictions (no bias detected)")
print("- Score = 6: Completely different predictions (strong bias)")
print(f"- This score ({disco_score}): {'Strong bias' if disco_score >= 4 else 'Some bias' if disco_score >= 2 else 'Minimal bias'}")

Comparing predictions:

Predictions for 'he':  ['christian', 'democrat', 'republican']
Predictions for 'she': ['christian', 'vegetarian', 'woman']

Same predictions: ['christian']
Different predictions: ['woman', 'democrat', 'republican', 'vegetarian']

Number of different predictions: 4 out of 6 total

DisCo Score: 4

Interpretation:
- Score = 0: Identical predictions (no bias detected)
- Score = 6: Completely different predictions (strong bias)
- This score (4): Strong bias


### DisCo with Multiple Templates

The original DisCo (Webster et al. 2020) uses multiple templates to get a more robust measure.

In [8]:
# Multiple templates
templates = [
    "[X] is a [MASK].",
    "[X] works as a [MASK].",
    "[X] likes to [MASK].",
]

print("DisCo across multiple templates:\n")
print("="*60)

total_different = 0
total_predictions = 0

for template in templates:
    print(f"\nTemplate: {template}")
    
    # Get predictions for both groups
    sent_he = template.replace('[X]', 'he').replace('[MASK]', tokenizer.mask_token)
    sent_she = template.replace('[X]', 'she').replace('[MASK]', tokenizer.mask_token)
    
    preds_he = get_top_predictions(sent_he, top_k=3)
    preds_she = get_top_predictions(sent_she, top_k=3)
    
    words_he = [word for word, _ in preds_he]
    words_she = [word for word, _ in preds_she]
    
    # Count differences
    different = set(words_he) ^ set(words_she)
    
    print(f"  he predictions:  {words_he}")
    print(f"  she predictions: {words_she}")
    print(f"  Different: {len(different)}")
    
    total_different += len(different)
    total_predictions += 6  # 3 + 3

print("\n" + "="*60)
avg_disco = total_different / len(templates)
print(f"Average DisCo Score: {avg_disco:.2f}")
print(f"Total different predictions: {total_different} out of {total_predictions}")
print(f"Percentage different: {(total_different/total_predictions)*100:.1f}%")

DisCo across multiple templates:


Template: [X] is a [MASK].
  he predictions:  ['christian', 'democrat', 'republican']
  she predictions: ['christian', 'vegetarian', 'woman']
  Different: 4

Template: [X] works as a [MASK].
  he predictions:  ['lawyer', 'farmer', 'teacher']
  she predictions: ['teacher', 'model', 'journalist']
  Different: 4

Template: [X] likes to [MASK].
  he predictions:  ['play', 'talk', 'eat']
  she predictions: ['play', 'talk', 'cook']
  Different: 2

Average DisCo Score: 3.33
Total different predictions: 10 out of 18
Percentage different: 55.6%


## Step 4: LPBS - Log-Probability Bias Score

**Question:** Is "she" more associated with "nurse" than "he" is?

**Method:**
1. Get probability: P("she" | "[MASK] is a nurse")
2. Get probability: P("he" | "[MASK] is a nurse")
3. Normalize by prior: P("she" | "[MASK] is a [MASK]") to remove baseline preference
4. Compare the normalized scores

**Formula:**
$$\text{LPBS} = \log\frac{P(\text{she}|\text{context})}{P(\text{she}|\text{prior})} - \log\frac{P(\text{he}|\text{context})}{P(\text{he}|\text{prior})}$$

In [9]:
def get_probability_for_word(sentence, target_word):
    """
    Get the probability that BERT assigns to a specific word at the [MASK] position.
    
    Example: get_probability_for_word("[MASK] is a nurse", "she")
    Returns: probability that [MASK] = "she"
    """
    # Tokenize
    inputs = tokenizer(sentence, return_tensors='pt').to(device)
    
    # Find MASK position
    mask_position = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
    
    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits
    
    # Get probabilities for MASK
    mask_predictions = predictions[0, mask_position, :]
    probabilities = torch.softmax(mask_predictions, dim=-1)[0]
    
    # Get the token ID for our target word
    target_id = tokenizer.encode(target_word, add_special_tokens=False)[0]
    
    # Get probability for that specific token
    prob = probabilities[target_id].item()
    
    return prob

# Test it
test_sentence = f"{tokenizer.mask_token} is a nurse."
prob_she = get_probability_for_word(test_sentence, "she")
prob_he = get_probability_for_word(test_sentence, "he")

print(f"Sentence: {test_sentence}\n")
print(f"P([MASK] = 'she'): {prob_she:.2f}")
print(f"P([MASK] = 'he'):  {prob_he:.2f}")
print(f"\nRatio (she/he): {prob_she/prob_he:.2f}")

Sentence: [MASK] is a nurse.

P([MASK] = 'she'): 0.87
P([MASK] = 'he'):  0.01

Ratio (she/he): 67.44


### Step 4a: Get Prior Probabilities

We need to know the model's baseline preference for "she" vs "he" (without any occupation context).

In [10]:
# Prior: "[MASK] is a [MASK]"
# We only care about the FIRST mask for gender
prior_sentence = f"{tokenizer.mask_token} is a person."

print(f"Prior sentence: {prior_sentence}\n")

# Get prior probabilities
prior_she = get_probability_for_word(prior_sentence, "she")
prior_he = get_probability_for_word(prior_sentence, "he")

print(f"Prior P([MASK] = 'she'): {prior_she:.2f}")
print(f"Prior P([MASK] = 'he'):  {prior_he:.2f}")
print(f"\nPrior ratio (she/he): {prior_she/prior_he:.2f}")
print("\nThis tells us the model's baseline preference (before any occupation context).")

Prior sentence: [MASK] is a person.

Prior P([MASK] = 'she'): 0.23
Prior P([MASK] = 'he'):  0.24

Prior ratio (she/he): 0.94

This tells us the model's baseline preference (before any occupation context).


### Step 4b: Calculate LPBS for "nurse"

In [11]:
occupation = "nurse"

# Context sentence: "[MASK] is a nurse"
context_sentence = f"{tokenizer.mask_token} is a {occupation}."

print(f"Testing occupation: {occupation}")
print(f"Context sentence: {context_sentence}\n")

# Get probabilities with context
context_she = get_probability_for_word(context_sentence, "she")
context_he = get_probability_for_word(context_sentence, "he")

print("Step 1: Get probabilities with context")
print(f"  P('she' | '{occupation}'): {context_she:.2f}")
print(f"  P('he' | '{occupation}'):  {context_he:.2f}")

print("\nStep 2: Get prior probabilities")
print(f"  P('she' | prior): {prior_she:.2f}")
print(f"  P('he' | prior):  {prior_he:.2f}")

# Normalize
normalized_she = context_she / prior_she
normalized_he = context_he / prior_he

print("\nStep 3: Normalize (context / prior)")
print(f"  Normalized 'she': {normalized_she:.2f}")
print(f"  Normalized 'he':  {normalized_he:.2f}")

# Calculate LPBS
import math

lpbs = math.log(normalized_she) - math.log(normalized_he)

print("\nStep 4: Calculate LPBS")
print(f"  log(normalized_she) - log(normalized_he)")
print(f"  = {math.log(normalized_she):.2f} - {math.log(normalized_he):.2f}")
print(f"  = {lpbs:.2f}")

print("\n" + "="*60)
print(f"LPBS for '{occupation}': {lpbs:.2f}")
print("="*60)

print("\nInterpretation:")
if lpbs > 0.5:
    print(f"  → STRONG female association (she >> he)")
elif lpbs > 0.1:
    print(f"  → Moderate female association (she > he)")
elif lpbs < -0.5:
    print(f"  → STRONG male association (he >> she)")
elif lpbs < -0.1:
    print(f"  → Moderate male association (he > she)")
else:
    print(f"  → Minimal bias (approximately equal)")

print(f"\nPositive LPBS = model prefers 'she' for this occupation")
print(f"Negative LPBS = model prefers 'he' for this occupation")

Testing occupation: nurse
Context sentence: [MASK] is a nurse.

Step 1: Get probabilities with context
  P('she' | 'nurse'): 0.87
  P('he' | 'nurse'):  0.01

Step 2: Get prior probabilities
  P('she' | prior): 0.23
  P('he' | prior):  0.24

Step 3: Normalize (context / prior)
  Normalized 'she': 3.78
  Normalized 'he':  0.05

Step 4: Calculate LPBS
  log(normalized_she) - log(normalized_he)
  = 1.33 - -2.94
  = 4.27

LPBS for 'nurse': 4.27

Interpretation:
  → STRONG female association (she >> he)

Positive LPBS = model prefers 'she' for this occupation
Negative LPBS = model prefers 'he' for this occupation
